In [1]:
import numpy as np
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver


from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import csv
from time import sleep

In [2]:
# Web Driver Configuration:
driver = webdriver.Chrome(ChromeDriverManager().install())

<ipython-input-2-ce91df882cb8>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
# Navigate to Amazon Page:
url = 'https://www.amazon.com/s?bbn=283155&rh=n%3A283155%2Cp_n_publication_date%3A1250226011&dc&qid=1724864554&rnid=1250225011&ref=lp_1000_nr_p_n_publication_date_0'
max_retries = 3  # Number of retries if something goes wrong

#Selenium is encountering a temporary issue when loading the page. So I implement a try-except block to automatically refresh and retry loading the page when the error occurs.

for attempt in range(max_retries):
    try:
        driver.get(url)
        time.sleep(5)  # Wait for the page to load
        if "Sorry" in driver.title:  # Check if there is an error in the title
            raise Exception("Page did not load correctly.")
        print(driver.title)  # Print the title if everything is fine
        break  # Exit the loop if successful
    except Exception as e:
        print(f"Attempt {attempt+1} failed: {e}")
        if attempt + 1 == max_retries:
            print("Max retries reached. Exiting.")
        else:
            print("Retrying...")
            time.sleep(5)  # Wait before retrying

Attempt 1 failed: Page did not load correctly.
Retrying...
Amazon.com


# scraping the first page 

In [4]:
html = driver.page_source
html

'<html lang="en-us" class=" a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.24.7-2024-09-09"><!-- sp:feature:head-start --><head><script async="" src="https://c.amazon-adsystem.com/bao-csm/forensics/a9-tq-forensics-incremental.min.js" crossorigin="anonymous"></script><script async="" src="https://images-na.ssl-images-amazon.com/images/I/31bJewCvY-L.js" crossorigin="anonymous"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8">\n<!-- sp:end-feature:head-start -->\n<!-- sp:feature:csm:head-open-part1 -->\n\n<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:end-feature:csm:head-open-part1 -->\n<!-- sp:feature:cs-optimiz

In [5]:
html = driver.page_source

In [6]:
soup = BeautifulSoup(html)
soup.prettify()

'<html class="a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.24.7-2024-09-09" lang="en-us">\n <!-- sp:feature:head-start -->\n <head>\n  <script async="" crossorigin="anonymous" src="https://c.amazon-adsystem.com/bao-csm/forensics/a9-tq-forensics-incremental.min.js">\n  </script>\n  <script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/31bJewCvY-L.js">\n  </script>\n  <script>\n   var aPageStart = (new Date()).getTime();\n  </script>\n  <meta charset="utf-8"/>\n  <!-- sp:end-feature:head-start -->\n  <!-- sp:feature:csm:head-open-part1 -->\n  <script type="text/javascript">\n   var ue_t0=ue_t0||+new Date();\n  </script>\n  <!-- sp:end-featu

In [7]:
containers = soup.find_all('div',{'class':'sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small gsx-ies-anchor sg-col-12-of-16'})
len(containers)

16

As indicated, this page contains 16 books (or reviews)

In [8]:
# let's try to extract informations from one book 
one = containers[4]
one

<div class="sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small gsx-ies-anchor sg-col-12-of-16" data-asin="1400260000" data-cel-widget="search_result_5" data-component-id="15" data-component-type="s-search-result" data-index="7" data-uuid="1b15bf89-236b-4291-b664-ec0d67a9a56d"><div class="sg-col-inner"><div cel_widget_id="MAIN-SEARCH_RESULTS-7" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_5" data-cel-widget="MAIN-SEARCH_RESULTS-7" data-csa-c-id="7rgz8i-im4myh-wnv9ss-1c9nz1" data-csa-c-item-id="amzn1.asin.1.1400260000" data-csa-c-pos="5" data-csa-c-type="item" data-csa-op-log-render=""><span class="a-declarative" data-action="puis-card-container-declarative" data-csa-c-func-deps="aui-da-puis-card-container-declarative" data-csa-c-id="o7oj81-lzz77b-q4fvrf-dh28ud" data-csa-c-item-id="amzn1.asin.1400260000" data-csa-c-owner="puis" data-csa-c-posx="5

In [9]:
title_recipe = one.find(attrs={"data-cy": "title-recipe"}).text
title_recipe

'What Happens Next: A Traveler’s Guide Through the End of This Age  by Max Lucado  | Aug 13, 2024'

we have the title the author and the date; not all the books have those data so we have to check if they're available to avoid errors.

In [10]:
if 'by' in title_recipe:
    parts = title_recipe.split('by')
    title = parts[0].strip()  # Extract title
    author_and_date = parts[1].strip()  # Everything after "by"
    
    # Check if "|" is in the second part to separate author and date
    if '|' in author_and_date:
        author, date = author_and_date.split('|')
        author = author.strip()
        date = date.strip()
    else:
        author = author_and_date.strip()  # No date available
        date = None
else:
    title = title_recipe.strip()  # Only title is available
    author = None
    date = None

In [11]:
title , author , date

('What Happens Next: A Traveler’s Guide Through the End of This Age',
 'Max Lucado',
 'Aug 13, 2024')

In [12]:
reviews_block = one.find(attrs={"data-cy": "reviews-block"}).text
reviews_block

'4.8 out of 5 stars 205  '

In [13]:
formats_elem = one.find_all('a', class_='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-bold')
price_elem = one.find_all('span', class_='a-price')

for f,p in zip(formats_elem,price_elem):
    title_recipe=one.find(attrs={"data-cy": "title-recipe"}).text
    # Extract book title
    if 'by' in title_recipe:
        parts = title_recipe.split('by')
        title = parts[0].strip()  # Extract title
        author_and_date = parts[1].strip()  # Everything after "by"

        # Check if "|" is in the second part to separate author and date
        if '|' in author_and_date:
            author, date = author_and_date.split('|')
            author = author.strip()
            date = date.strip()
        else:
            author = author_and_date.strip()  # No date available
            date = None
    else:
        title = title_recipe.strip()  # Only title is available
        author = None
        date = None
    # Extract rating
    rating = one.find('span', class_='a-icon-alt').text
    # Extract number of ratings
    num_ratings = one.find('span', class_='a-size-base s-underline-text').text

    # extract the badge (if existed)
    badge_tag = one.find('span', class_='a-badge-text')
    if badge_tag:
            badge = badge_tag.text
    else:
            badge = None

    # Extract book format and price
    book_format = f.text.strip()
    price = p.find('span', class_='a-offscreen').text.strip()

    print(f"Title: {title}")
    print(f"Author: {author}")
    print(f"Publication Date: {date}")
    print(f"Rating: {rating}")
    print(f"Number of Ratings: {num_ratings}")
    print(f"Format: {book_format}")
    print(f"Price: {price}")
    print(f"Badge: {badge}")
    print('---------------------------')

Title: What Happens Next: A Traveler’s Guide Through the End of This Age
Author: Max Lucado
Publication Date: Aug 13, 2024
Rating: 4.8 out of 5 stars
Number of Ratings: 205
Format: Hardcover
Price: $15.99
Badge: Best Seller
---------------------------
Title: What Happens Next: A Traveler’s Guide Through the End of This Age
Author: Max Lucado
Publication Date: Aug 13, 2024
Rating: 4.8 out of 5 stars
Number of Ratings: 205
Format: Audible Audiobook
Price: $29.99
Badge: Best Seller
---------------------------
Title: What Happens Next: A Traveler’s Guide Through the End of This Age
Author: Max Lucado
Publication Date: Aug 13, 2024
Rating: 4.8 out of 5 stars
Number of Ratings: 205
Format: Kindle
Price: $0.00
Badge: Best Seller
---------------------------


In [14]:
#Let's extract all the books 
reviews_list = []
for con in containers:
    
    formats_elem = con.find_all('a', class_='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-bold')
    price_elem = con.find_all('span', class_='a-price')

    for f,p in zip(formats_elem,price_elem):
        title_recipe=con.find(attrs={"data-cy": "title-recipe"}).text
        # Extract book title
        if 'by' in title_recipe:
            parts = title_recipe.split('by')
            title = parts[0].strip()  # Extract title
            author_and_date = parts[1].strip()  # Everything after "by"

            # Check if "|" is in the second part to separate author and date
            if '|' in author_and_date:
                author, date = author_and_date.split('|')
                author = author.strip()
                date = date.strip()
            else:
                author = author_and_date.strip()  # No date available
                date = None
        else:
            title = title_recipe.strip()  # Only title is available
            author = None
            date = None
        # Extract rating
        rating = con.find('span', class_='a-icon-alt').text
        # Extract number of ratings
        num_ratings = con.find('span', class_='a-size-base s-underline-text').text

        # extract the badge (if existed)
        badge_tag = con.find('span', class_='a-badge-text')
        if badge_tag:
                badge = badge_tag.text
        else:
                badge = None

        # Extract book format and price
        book_format = f.text.strip()
        price = p.find('span', class_='a-offscreen').text.strip()
        
        reviews_list.append({
                'Title': title,
                'Author':author,
                "Publication Date": date,
                "Rating": rating,
                "Number of Ratings": num_ratings,
                "Format": book_format,
                "Price": price,
                "Badge": badge })
                


let's define our funtion now 

In [38]:
all_books = []
def scrape_books(driver):
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html)
    soup.prettify()
    containers = soup.find_all('div',{'class':'sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small gsx-ies-anchor sg-col-12-of-16'})

    for con in containers:

        formats_elem = con.find_all('a', class_='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-bold')
        price_elem = con.find_all('span', class_='a-price')

        for f,p in zip(formats_elem,price_elem):
            title_recipe=con.find(attrs={"data-cy": "title-recipe"}).text
            # Extract book title
            if 'by' in title_recipe:
                parts = title_recipe.split('by')
                title = parts[0].strip()  # Extract title
                author_and_date = parts[1].strip()  # Everything after "by"

                # Check if "|" is in the second part to separate author and date
                if '|' in author_and_date:
                    author, date = author_and_date.split('|')
                    author = author.strip()
                    date = date.strip()
                else:
                    author = author_and_date.strip()  # No date available
                    date = None
            else:
                title = title_recipe.strip()  # Only title is available
                author = None
                date = None
            # Extract rating
            rating_icon = con.find('span', class_='a-icon-alt')
            if rating_icon:
                    rating = rating_icon.text
            else:
                    rating = None
            # Extract number of ratings
            num_ratings_zone= con.find('span', class_='a-size-base s-underline-text')
            if num_ratings_zone:
                    num_ratings = num_ratings_zone.text
            else:
                    num_ratings = None
            # extract the badge (if existed)
            badge_tag = con.find('span', class_='a-badge-text')
            if badge_tag:
                    badge = badge_tag.text
            else:
                    badge = None

            # Extract book format and price
            book_format = f.text.strip()
            price = p.find('span', class_='a-offscreen').text.strip()

            all_books.append({
                    'Title': title,
                    'Author':author,
                    "Publication Date": date,
                    "Rating": rating,
                    "Number of Ratings": num_ratings,
                    "Format": book_format,
                    "Price": price,
                    "Badge": badge })



# Navigating Through Pages

Click on pagination buttons to navigate through multiple pages of content

In [ ]:
while True:
    try :
        time.sleep(5)
        scrape_books(driver)
        next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
        next_button.click()
        
    except Exception as e:
        print(f"No more pages")
        break

In [53]:
len(all_books)

2623

In [54]:
df = pd.DataFrame(all_books)
df

,Title,Author,Publication Date,Rating,Number of Ratings,Format,Price,Badge
0,"The Dragon's Prophecy: Israel, the Dark Resurr...",Jonathan Cahn,"Sep 3, 2024",4.8 out of 5 stars,249,Hardcover,$15.49,Best Seller
1,"The Dragon's Prophecy: Israel, the Dark Resurr...",Jonathan Cahn,"Sep 3, 2024",4.8 out of 5 stars,249,Audible Audiobook,$28.99,Best Seller
2,"The Dragon's Prophecy: Israel, the Dark Resurr...",Jonathan Cahn,"Sep 3, 2024",4.8 out of 5 stars,249,Kindle,$0.00,Best Seller
3,Lovely One: A Memoir,Ketanji Brown Jackson,"Sep 3, 2024",4.6 out of 5 stars,40,Hardcover,$24.50,None
4,Lovely One: A Memoir,Ketanji Brown Jackson,"Sep 3, 2024",4.6 out of 5 stars,40,Audible Audiobook,$35.00,None
...,...,...,...,...,...,...,...,...
2618,"Leave and Cleave: Nine Marriage Prep Secrets, ...",Timothy J. Gordon and Stephanie C. Gordon,"Aug 31, 2024",5.0 out of 5 stars,1,Hardcover,$30.00,None
2619,"Anne of Green Gables: The Complete Novel, Feat...",Hand (Handwritten Classics),None,None,None,Hardcover,$33.44,None
2620,Speaking of America: United States Presidents ...,Jared Cohen and Vivian Shih,"Sep 10, 2024",5.0 out of 5 stars,2,Hardcover,$17.31,None
2621,Speaking of America: United States Presidents ...,Jared Cohen and Vivian Shih,"Sep 10, 2024",5.0 out of 5 stars,2,Audible Audiobook,$19.99,None


In [ ]:
df.to_excel('book-reviews.xlsx')